[1036. Escape a Large Maze](https://leetcode.com/problems/escape-a-large-maze/)

In [ ]:
public class SortedHashSet<T> : IEnumerable<T> where T : IComparable<T>
{
    private Dictionary<int, List<T>> _dict = new();
    private List<T> _list = new();

    public int Count => _list.Count;

    public void Add(T el)
    {
        var hash = el.GetHashCode();
        if(_dict.ContainsKey(hash))
        {
            if(_dict[hash].Contains(el))
                return;
        }
        else _dict[hash] = new();
        _dict[hash].Add(el);

        if(_list.Count == 0)
            _list.Add(el);
        else
        {
            int left = 0;
            int right = _list.Count;
            while(left != right)
            {
                int index = left + (right - left) / 2;
                if(_list[index].CompareTo(el) < 0)
                    left = index + 1;
                else right = index;
            }
            _list.Insert(left, el);
        }
    }

    public int IndexOf(T el)
    {
        int left = 0;
        int right = _list.Count;
        while(left != right)
        {
            int index = left + (right - left) / 2;
            int compare = _list[index].CompareTo(el);
            if(compare < 0)
                left = index + 1;
            else if(compare > 0)
                right = index;
            else
            {
                left = right = index;
                for(int i = index; i >= 0; i--)
                {
                    if(_list[i].Equals(el))
                        return i;
                    if(_list[i].CompareTo(el) != 0)
                        break;
                }
                for(int i = index + 1; i < _list.Count; i++)
                {
                    if(_list[i].Equals(el))
                        return i;
                    if(_list[i].CompareTo(el) != 0)
                        break;
                }
            }
        }
        return -1;
    }

    public bool Contains(T el)
    {
        var hash = el.GetHashCode();
        return _dict.ContainsKey(hash) && _dict[hash].Contains(el);
    }

    public bool TryGetEqual(T el, out T res)
    {
        var hash = el.GetHashCode();
        if(_dict.ContainsKey(hash))
        {
            int index = _dict[hash].IndexOf(el);
            if(index < 0)
            {
                res = default;
                return false;
            }
            else
            {
                res = _dict[hash][index];
                return true;
            }
        }
        else
        {
            res = default;
            return false;
        }
    }

    public void Remove(T el)
    {
        var hash = el.GetHashCode();
        if(_dict.ContainsKey(hash))
        {
            if(_dict[hash].Contains(el))
                _dict[hash].Remove(el);
            if(_dict[hash].Count == 0)
                _dict.Remove(hash);
        }
        var index = IndexOf(el);
        if(index >= 0)
            _list.RemoveAt(index);
    }

    public T Pop(int? index = null)
    {
        var ret = _list[index ?? _list.Count - 1];
        Remove(ret);
        return ret;
    }

    public IEnumerator<T> GetEnumerator()
        => _list.GetEnumerator();

    IEnumerator IEnumerable.GetEnumerator()
        => this.GetEnumerator();
}

public struct Element : IComparable<Element>
{
    public int X;
    public int Y;
    public int G;
    public int H;
    public int F;

    public int CompareTo(Element el)
        => F.CompareTo(el.F);

    public override bool Equals(object obj)
        => obj is Element el && X == el.X && Y == el.Y;

    public override int GetHashCode()
        => (X.GetHashCode() * 1000000) | Y.GetHashCode();

    public override string ToString()
        => $"X:{X} Y:{Y} F:{F}";

    public static bool operator ==(Element a, Element b)
        => a.Equals(b);

    public static bool operator !=(Element a, Element b)
        => !a.Equals(b);
}

public class Solution
{
    private const int MAP_SIZE = 1_000_000;
    private static int _i = 0;
    private static int minX, maxX, minY, maxY;
    
    private int _startX;
    private int _startY;
    private int _endX;
    private int _endY;
    
    private HashSet<(int x, int y)> _blocked = new();
    private SortedHashSet<Element> _open = new();
    private HashSet<Element> _closed = new();
    
    public bool IsEscapePossible(int[][] blocked, int[] source, int[] target, bool reverse = false)
    {
        if(blocked.Length == 0)
            return true;

        minX = minY = int.MaxValue;
        maxX = maxY = 0;
        
        var compare = (int[] arr) =>
        {
            minX = Math.Min(minX, arr[0]);
            maxX = Math.Max(maxX, arr[0]);
            minY = Math.Min(minY, arr[1]);
            maxY = Math.Max(maxY, arr[1]);
        };
        
        foreach(var pos in blocked) compare(pos);
        compare(source);
        compare(target);
        
        _startX = source[0];
        _startY = source[1];
        _endX = target[0];
        _endY = target[1];
        
        foreach(var pos in blocked)
            _blocked.Add((pos[0], pos[1]));
        
        var startEl = new Element
        {
            X = _startX,
            Y = _startY,
            G = 0,
            H = CalcH(_startX, _startY),
            F = CalcH(_startX, _startY)
        };
        _open.Add(startEl);
        
        bool find = false;

        while(_open.Count > 0)
        {
            var el = _open.Pop(0);

            // HACK that use fack that there are can be only 200 blocked cells :nodder
            if(Math.Abs(el.X - _startX) + Math.Abs(el.Y - _startY) > 400)
            {
                if(reverse)
                {
                    find = true;
                    break;
                }

                _blocked = new();
                _open = new();
                _closed = new();
                find = IsEscapePossible(blocked, target, source, true);
                break;
            }
            // END OF HACK
            
            if(el.X == _endX && el.Y == _endY)
            {
                find = true;
                break;
            }

            var moves = GetPossibleMoves(el);

            foreach(var move in moves)
            {
                if(_open.TryGetEqual(move, out var open) && open.F <= move.F)
                    continue;
                if(_closed.TryGetValue(move, out var closed) && closed.F <= move.F)
                    continue;

                _open.Add(move);
            }

            _closed.Remove(el);
            _closed.Add(el);
        }
        
        return find;
    }

    private List<Element> GetPossibleMoves(Element el)
    {
        var ret = new List<Element>();

        if(el.X + 1 < MAP_SIZE && !_blocked.Contains((el.X + 1, el.Y)))
        {
            var h = CalcH(el.X + 1, el.Y);
            ret.Add(new Element
            {
                X = el.X + 1,
                Y = el.Y,
                G = el.G + 1,
                H = h,
                F = el.G + 1 + h
            });
        }
        
        if(el.X - 1 >= 0 && !_blocked.Contains((el.X - 1, el.Y)))
        {
            var h = CalcH(el.X - 1, el.Y);
            ret.Add(new Element
            {
                X = el.X - 1,
                Y = el.Y,
                G = el.G + 1,
                H = h,
                F = el.G + 1 + h
            });
        }
        
        if(el.Y + 1 < MAP_SIZE && !_blocked.Contains((el.X, el.Y + 1)))
        {
            var h = CalcH(el.X, el.Y + 1);
            ret.Add(new Element
            {
                X = el.X,
                Y = el.Y + 1,
                G = el.G + 1,
                H = h,
                F = el.G + 1 + h
            });
        }
        
        if(el.Y - 1 >= 0 && !_blocked.Contains((el.X, el.Y - 1)))
        {
            var h = CalcH(el.X, el.Y - 1);
            ret.Add(new Element
            {
                X = el.X,
                Y = el.Y - 1,
                G = el.G - 1,
                H = h,
                F = el.G + 1 + h
            });
        }

        return ret;
    }

    private int CalcH(int x, int y)
        => Math.Abs(x - _endX) + Math.Abs(y - _endY);
}